In [1]:
from PIL import Image
import numpy as np 
import os
import random
from collections import Counter

In [21]:
path_1 = "train_700_28\\truth.dsv"
path_2 = "train_1000_10\\truth.dsv"
path_3 = "train_1000_28\\truth.dsv"
__file__ = "p1"

In [102]:
def get_true_output_training_sets(path): 
    my_path = os.path.abspath(os.path.dirname(__file__))
    path = os.path.join(my_path, path)
    output_dict = {}
    with open(path) as f:
        lines = f.readlines()
        for line_index in range(len(lines)): 
            after_semi = lines[line_index].partition(":")[2]
            pic_id = lines[line_index].partition(":")[0]
            true_output = after_semi[0]
            if true_output not in output_dict: 
                output_dict[true_output] = [pic_id]
            else: 
                output_dict[true_output].append(pic_id)
            #output_dict[pic_id] = true_output
    return output_dict

def randomize(output_dict): 
    l = list(output_dict.items())
    random.shuffle(l)
    d = dict(l)
    return d

In [87]:
def get_files(path): 
    for fname in os.listdir(path):
        print(fname)

def test_image_handling(image_path, true_values_dict): 
    for key in true_values_dict: 
        my_path = os.path.abspath(os.path.dirname(__file__))
        path = os.path.join(my_path, image_path + "\\" + key)
        im = Image.open(path)
        #im = Image.open(image_path+ "\\" + key)
        print(type(im))
        im2d = np.array(im)
        im1d = im2d.flatten()
        return im, im2d, im1d

def get_distance(picture_1, picture_2): 
    #Her bør nok bildet endres litt før det kommer inn. Kanskje prøve å gjøre en slags max pooling først?
    
    im_1 = Image.open(picture_1)
    im_2 = Image.open(picture_2)
    im_1 = im_1.resize((5, 5),Image.ANTIALIAS)
    im_2 = im_2.resize((5, 5),Image.ANTIALIAS)
    im1 = np.array(im_1).flatten()
    im2 = np.array(im_2).flatten()
    diff = im1 - im2
    d2 = np.linalg.norm(diff)
    return d2 

def get_most_likely_value(neighbours_list): 
    #taken from here: https://www.geeksforgeeks.org/python-find-most-frequent-element-in-a-list/
    occurence_count = Counter(neighbours_list)
    return occurence_count.most_common(1)[0][0]


In [5]:
def k_NN_train(training_dictionary, k, path): 
    #noe galt her, men den over funker så se på den
    correct_value = 0 
    for test_pic in training_dictionary: 
        best_neighbours_list = [np.inf for _ in range(k*3)]
        for key, item in training_dictionary.items(): 
            if key != test_pic:
                eucledian_distance = get_distance((path + "\\" + test_pic), (path + "\\" + key))
                for best_neighbour_index in range(2, len(best_neighbours_list), 3):
                    if eucledian_distance < best_neighbours_list[best_neighbour_index] and (key not in best_neighbours_list): 
                        best_neighbours_list[best_neighbour_index-2] = key
                        best_neighbours_list[best_neighbour_index-1] = item
                        best_neighbours_list[best_neighbour_index] = eucledian_distance
        
        most_likely_value = get_most_likely_value(best_neighbours_list[1::2])
        #denna funke ikke og gir feil svar?? 
        if(most_likely_value == training_dictionary[test_pic]): 
            correct_value+=1
        #training_dictionary[test_pic] = [training_dictionary[test_pic], most_likely_value]
    return correct_value/len(training_dictionary)  

In [136]:
def classes_probabilities(training_dictionary): 
    total_probs = {}
    total = 0
    for key, item in training_dictionary.items(): 
        total_probs[key] = len(item)
        total += len(item)
    for key, item in total_probs.items(): 
        total_probs[key] = item/total
    return total_probs

def get_image_proportions(picture): 
    im = Image.open(picture)
    return im, im.width



def minimize_pic(picture, picture_width):
    im = np.array(picture)
    flattened_im = im.flatten()
    minimized_im = np.zeros(np.size(flattened_im)//picture_width)
    for pixel in range(picture_width,np.size(flattened_im),picture_width): 
        pixel_average = np.average(flattened_im[pixel-picture_width:pixel+1])
        pixel_feature = features_in_pic(pixel_average)
        minimized_im[(pixel//picture_width)-1] = pixel_feature 
    return minimized_im
    
def features_in_pic(pixel_average): 
    if(pixel_average < 256*1/8):
        pixel_class = 1
    elif pixel_average < 256*2/8:
        pixel_class = 2
    elif pixel_average < 256*3/8:
        pixel_class = 3
    elif pixel_average < 256*4/8:
        pixel_class = 4
    elif pixel_average < 256*5/8:
        pixel_class = 5
    elif pixel_average < 256*6/8:
        pixel_class = 6
    elif pixel_average < 256*7/8:
        pixel_class = 7
    else: 
        pixel_class = 8  
    return pixel_class

def get_conditional_probs(classes_probabilities):
    """
    Vet at jeg har 8 pikselklasser, så må ha 8 conditional probabilities for hver ting jeg kan klassifisere som. 
    """
    return None 

In [137]:
def get_simplified_pic_output_value(out_dictionary): 
    for _, picture_list in out_dictionary.items():
        for picture_index in range(len(picture_list)): 
            picture, picture_width = get_image_proportions("train_1000_28\\" + picture_list[picture_index])
            minimized_picture = minimize_pic(picture, picture_width)
            picture_list[picture_index] = minimized_picture
    return out_dictionary
    

In [7]:
def naive_bayes_train(): 
    """
    1. Få dictionaryen fra treningssettet og finn ut hvor stor prosent de ulike typene er av den totale summen.
        Kanskje bruk sett for å fikse? - DONE 
    2. Fiks bildene ved feks bruk av max eller average pooling og gjør de mindre. De nye pikslene vil være featursene. - DONE
    3. Finn et intervall mellom 0 og 255 som hver utgjør en feature. Du kan jo egt bare endre bildene akkurat som du vil, 
        feks bare dele i sånn 8 klasser (ish 32 pikslelverdier på hver da) - DONE
    4. Kanskje bare kalle hver piksel et tall mellom 1 og 8?  DONE
    5. Gå gjennom bildene og finn sannsynlighet for at et feature er i et tall gitt at det tallet er et tall. Husk uavhengighet! 
    """
    return None
def naive_bayes_test(): 
    return None 

In [138]:
true_values_dict = get_true_output_training_sets(path_1)
test_dict = randomize(true_values_dict)
test_count = classes_probabilities(test_dict)
_, b = get_image_proportions("train_1000_28\\img_0001.png")
print(test_count)
test = get_simplified_pic_output_value(test_dict)
print(test)
#print(os.listdir("train_1000_10"))
#print(get_files(os.path.abspath(os.path.dirname("train_1000_10"))))


28
{'7': 0.03571428571428571, '0': 0.14285714285714285, '1': 0.14285714285714285, '9': 0.03571428571428571, '8': 0.03571428571428571, '3': 0.14285714285714285, '6': 0.03571428571428571, '4': 0.14285714285714285, '5': 0.14285714285714285, '2': 0.14285714285714285}
{'7': [array([8., 8., 8., 8., 8., 8., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7.,
       5., 7., 8., 8., 8., 8., 8., 8., 8., 8., 0.]), array([8., 8., 8., 8., 8., 7., 6., 7., 7., 8., 8., 8., 8., 8., 7., 6., 5.,
       6., 6., 7., 7., 6., 6., 8., 8., 8., 8., 0.]), array([8., 8., 8., 8., 8., 8., 7., 7., 7., 7., 7., 7., 7., 7., 7., 6., 6.,
       8., 8., 8., 8., 8., 8., 8., 8., 8., 8., 0.]), array([8., 8., 8., 8., 8., 8., 8., 6., 7., 8., 8., 7., 7., 7., 8., 8., 8.,
       7., 7., 6., 6., 8., 8., 8., 8., 8., 8., 0.]), array([8., 8., 8., 8., 8., 8., 6., 6., 6., 6., 7., 7., 7., 6., 7., 8., 8.,
       8., 7., 7., 6., 6., 6., 7., 8., 8., 8., 0.]), array([8., 8., 8., 8., 7., 7., 6., 6., 7., 7., 7., 7., 7., 7., 7., 7., 7.,
       7., 7.,